In [4]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.executor.instances', '8')\
    .config('spark.jars', '/opt/dev/target/ml-on-spark-1.0.jar')\
    .getOrCreate()

#   
    
print('pyspark ready ...')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [5]:
train = spark.read.format("parquet").load("/data/books/ratings-train.parquet")
train.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [6]:
%autoreload
from mlonspark.alternating_least_square import AlternatingLeastSquare

als = AlternatingLeastSquare()\
    .setUserCol("userId")\
    .setItemCol("bookId")\
    .setRatingCol("rating")

print(als)

AlternatingLeastSquare_48c2ada374dbac9b6437


In [7]:
model = als.fit(train)
print(model)

AlternatingLeastSquare_48c2ada374dbac9b6437


In [8]:
model.write().overwrite().save('/data/books/model.alsmodel')

In [9]:
%autoreload
from mlonspark.alternating_least_square import AlternatingLeastSquareModel

model = AlternatingLeastSquareModel.read().load('/data/books/model.alsmodel')

In [10]:
print(model)

AlternatingLeastSquare_48c2ada374dbac9b6437


In [11]:
test = spark.read.format("parquet").load("/data/books/ratings-test.parquet")
test.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [12]:
predictions = model.transform(test)
predictions.printSchema()
predictions.show(10)

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- prediction: float (nullable = false)

+-------+----------+-----------+------+------+--------------------+---+------+------------+
|User-ID|      ISBN|Book-Rating|rating|bookId|            Location|Age|userId|  prediction|
+-------+----------+-----------+------+------+--------------------+---+------+------------+
| 100004|0345339703|         10|   4.0|  1571|san ysidro, calif...|  0|100004|  0.05808821|
| 100004|0440414806|          0|   1.0|  5649|san ysidro, calif...|  0|100004| 0.023661885|
| 100004|155874942X|          0|   1.0|111016|san ysidro, calif...|  0|100004|5.0911435E-4|
| 100009|0060392452|          8|   2.0|  2802|victoria, british...| 49|100009| 0.025896763|


In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import isnan

evaluator = RegressionEvaluator()\
    .setMetricName("rmse")\
    .setLabelCol("rating")\
    .setPredictionCol("prediction")

predictionsFiltered = predictions.where(~isnan(col("prediction")))
                                
print("number of predictions = %i" % predictionsFiltered.count())

rmse = evaluator.evaluate(predictionsFiltered)

print("RMSE = %f" % rmse)

number of predictions = 279936
RMSE = 1.621634


In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.recommendation import ALS
from mlonspark.alternating_least_square import AlternatingLeastSquare

als2 = ALS()\
    .setUserCol("userId")\
    .setItemCol("bookId")\
    .setRatingCol("rating")

pipeline = Pipeline().setStages([als])

params = ParamGridBuilder()\
    .build()
#    .addGrid(als.rank, [2, 5, 10])\
    

cv = CrossValidator()\
    .setEstimator(als)\
    .setEstimatorParamMaps(params)\
    .setEvaluator(evaluator)\
    .setNumFolds(5)

model = cv.fit(train)

In [ ]:
model.bestModel
model.avgMetrics

In [ ]:
model.write().overwrite().save('/data/books/crossvalidation.model')